In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from scipy.sparse.linalg import cg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import os
import sys
import glob
import time
import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.tree 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import pandas as pd



In [2]:
# 导入数据

data = pd.read_csv('C:/Jiqiang/jh_z/jh_z/model training/final_data_input_important.csv')
df= pd.DataFrame(data)
df
# 选择300个label=1的行
df_label_1 = df[df['label'] == 1].sample(n=1500)

# 选择300个label=0的行
df_label_0 = df[df['label'] == 0].sample(n=1500)

# 将两个数据框连接成一个新数据框
df_train_test = pd.concat([df_label_1, df_label_0])
df_train_test

,FID_,average_max_rainfall,average_2hr_rainfall,average_72hr_rainfall,distance to coast,EV,TWI,DTW,label
8869,8987,1.043519,0.161838,0.547632,892.030195,2.820148,138.425158,7.398143,1
8988,9107,0.960301,0.168526,0.559966,1148.893755,3.171038,3456.031638,4.513801,1
12675,12840,0.953163,0.162185,0.553570,49.989193,-0.900000,8.288717,19.147230,1
14604,14797,1.070443,0.155097,0.557052,1158.937154,2.691030,1705.613215,7.212203,1
8734,8851,1.029160,0.165419,0.551839,566.903276,3.099591,1434.147206,6.207119,1
...,...,...,...,...,...,...,...,...,...
14002,14186,1.073781,0.169438,0.600820,157.836087,2.414823,601.221331,4.187641,0
5636,5703,0.922260,0.150654,0.528119,75.633159,2.290039,302.489656,4.815857,0
10594,10725,0.968089,0.166842,0.560369,950.013772,2.240190,3394.595840,7.077906,0
1455,1472,0.986703,0.174033,0.550543,359.680910,2.841286,197.297653,6.094360,0


In [3]:
# 提取最后一列并命名为“y”
Y = df_train_test.iloc[:, -1]
Y.name = 'y'

# 提取除最后一列以外的所有列并命名为“X”
X = df_train_test.iloc[:, 1:-1]


In [4]:
X

,average_max_rainfall,average_2hr_rainfall,average_72hr_rainfall,distance to coast,EV,TWI,DTW
8869,1.043519,0.161838,0.547632,892.030195,2.820148,138.425158,7.398143
8988,0.960301,0.168526,0.559966,1148.893755,3.171038,3456.031638,4.513801
12675,0.953163,0.162185,0.553570,49.989193,-0.900000,8.288717,19.147230
14604,1.070443,0.155097,0.557052,1158.937154,2.691030,1705.613215,7.212203
8734,1.029160,0.165419,0.551839,566.903276,3.099591,1434.147206,6.207119
...,...,...,...,...,...,...,...
14002,1.073781,0.169438,0.600820,157.836087,2.414823,601.221331,4.187641
5636,0.922260,0.150654,0.528119,75.633159,2.290039,302.489656,4.815857
10594,0.968089,0.166842,0.560369,950.013772,2.240190,3394.595840,7.077906
1455,0.986703,0.174033,0.550543,359.680910,2.841286,197.297653,6.094360


In [5]:
Y

8869     1
8988     1
12675    1
14604    1
8734     1
        ..
14002    0
5636     0
10594    0
1455     0
15139    0
Name: y, Length: 3000, dtype: int64

In [6]:
# 分层采样方法分割数据集：
X_train, X_test,Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y)


In [7]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

print(pd.value_counts(Y_train))

print(pd.value_counts(Y_test))

(2400, 7) (600, 7) (2400,) (600,)
1    1200
0    1200
Name: y, dtype: int64
1    300
0    300
Name: y, dtype: int64


In [8]:
def print_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

shap.initjs()

In [9]:
# SVM
svc_linear = sklearn.svm.SVC(kernel='linear', probability=True)
svc_linear.fit(X_train, Y_train)
print_accuracy(svc_linear.predict)
svc_linear.predict(X_test)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(svc_linear.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

Accuracy = 73.0%


X does not have valid feature names, but SVC was fitted with feature names
Using 2400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/600 [00:00<?, ?it/s]

X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid feature names, but SVC was fitted with feature names
X does not have valid fea

In [31]:
# y_true：测试集的真实标签
# y_pred：测试集的预测标签

# SVM模型性能评估
# 准确率
y_true = Y_test
y_pred_svm = svc_linear.predict(X_test)
accuracy_svm = accuracy_score(y_true, y_pred_svm)

# 精确度
precision_svm = precision_score(y_true, y_pred_svm)

# 召回率
recall_svm = recall_score(y_true, y_pred_svm)

# F1分数
f1_svm = f1_score(y_true, y_pred_svm)

# ROC-AUC分数
roc_auc_svm = roc_auc_score(y_true, y_pred_svm)

print(accuracy_svm, precision_svm, recall_svm, f1_svm, roc_auc_svm)

0.73 0.73 0.73 0.7299999999999999 0.73


In [11]:
# Logistic regression
linear_lr = sklearn.linear_model.LogisticRegression()
linear_lr.fit(X_train, Y_train)
print_accuracy(linear_lr.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(linear_lr.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)


Accuracy = 73.0%


X does not have valid feature names, but LogisticRegression was fitted with feature names
Using 2400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/600 [00:00<?, ?it/s]

X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not have valid feature names, but LogisticRegression was fitted with feature names
X does not

In [32]:
# LR模型性能评估
# 准确率
y_true = Y_test
y_pred_lr = linear_lr.predict(X_test)
accuracy_lr = accuracy_score(y_true, y_pred_lr)

# 精确度
precision_lr = precision_score(y_true, y_pred_lr)

# 召回率
recall_lr = recall_score(y_true, y_pred_lr)

# F1分数
f1_lr = f1_score(y_true, y_pred_lr)

# ROC-AUC分数
roc_auc_lr = roc_auc_score(y_true, y_pred_lr)

print(accuracy_lr, precision_lr, recall_lr, f1_lr, roc_auc_lr)

0.73 0.7363013698630136 0.7166666666666667 0.7263513513513512 0.7300000000000001


In [13]:
# DT
import sklearn.tree 
dtree = sklearn.tree.DecisionTreeClassifier(min_samples_split=2)
dtree.fit(X_train, Y_train)
print_accuracy(dtree.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(dtree.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

Accuracy = 69.16666666666667%


X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
Using 2400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/600 [00:00<?, ?it/s]

X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
X does not have valid feature names, but DecisionTreeClassif

In [33]:
# DT模型性能评估
# 准确率
y_true = Y_test
y_pred_dt = dtree.predict(X_test)
accuracy_dt = accuracy_score(y_true, y_pred_dt)

# 精确度
precision_dt = precision_score(y_true, y_pred_dt)

# 召回率
recall_dt = recall_score(y_true, y_pred_dt)

# F1分数
f1_dt = f1_score(y_true, y_pred_dt)

# ROC-AUC分数
roc_auc_dt = roc_auc_score(y_true, y_pred_dt)

print(accuracy_dt, precision_dt, recall_dt, f1_dt, roc_auc_dt)

0.6916666666666667 0.6910299003322259 0.6933333333333334 0.692179700499168 0.6916666666666667


In [15]:
# RF
from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
rforest.fit(X_train, Y_train)
print_accuracy(rforest.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(rforest.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

Accuracy = 78.83333333333333%


X does not have valid feature names, but RandomForestClassifier was fitted with feature names
Using 2400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/600 [00:00<?, ?it/s]

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassif

In [34]:
# RF模型性能评估
# 准确率
y_true = Y_test
y_pred_rf = rforest.predict(X_test)
accuracy_rf = accuracy_score(y_true, y_pred_rf)

# 精确度
precision_rf = precision_score(y_true, y_pred_rf)

# 召回率
recall_rf = recall_score(y_true, y_pred_rf)

# F1分数
f1_rf = f1_score(y_true, y_pred_rf)

# ROC-AUC分数
roc_auc_rf = roc_auc_score(y_true, y_pred_rf)

print(accuracy_rf, precision_rf, recall_rf, f1_rf, roc_auc_rf)

0.7883333333333333 0.7952218430034129 0.7766666666666666 0.7858347386172007 0.7883333333333333


In [23]:
# XGBoost
'''
xgb_train = xgboost.DMatrix(X_train, label=Y_train)
xgb_test = xgboost.DMatrix(X_test, label=Y_test)

# use validation set to choose # of trees
params = {
    "eta": 0.002,
    "max_depth": 3,
    # "objective": "survival:cox",
    "subsample": 0.5
}
model_train = xgboost.train(params, xgb_train, 10000, evals = [(xgb_test, "test")], verbose_eval=1000)

# CHEK PERFORMACE
def statistic(pred, labels):
    total = 0
    matches = 0
    for i in range(len(labels)):
        for j in range(len(labels)):
            if labels[j] > 0 and abs(labels[i]) > labels[j]:
                total += 1
                if pred[j] > pred[i]:
                    matches += 1
    return matches/total

# see how well we can order people by survival
# statistic(model_train.predict(xgb_test, ntree_limit=5000), Y_test)
statistic(model_train.predict(xgb_test), Y_test)
'''

xgb_model = xgboost.XGBClassifier(objective='binary:logistic')
xgb_model.fit(X_train,Y_train)
print_accuracy(xgb_model.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(xgb_model.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)


Accuracy = 77.83333333333333%


Using 2400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/600 [00:00<?, ?it/s]

In [35]:
# xgboost模型性能评估
# 准确率
y_true = Y_test
y_pred_xgb = xgb_model.predict(X_test)
accuracy_xgb = accuracy_score(y_true, y_pred_xgb)

# 精确度
precision_xgb = precision_score(y_true, y_pred_xgb)

# 召回率
recall_xgb = recall_score(y_true, y_pred_xgb)

# F1分数
f1_xgb = f1_score(y_true, y_pred_xgb)

# ROC-AUC分数
roc_auc_xgb = roc_auc_score(y_true, y_pred_xgb)

print(accuracy_xgb, precision_xgb, recall_xgb, f1_xgb, roc_auc_xgb)

0.7783333333333333 0.7755775577557755 0.7833333333333333 0.7794361525704809 0.7783333333333332


In [ ]:
shap.summary_plot(shap_values, X)
# shap.dependence_plot("Age", shap_values, X)
#shap.dependence_plot("Sex", shap_values, X, display_features=X_display)
# shap.dependence_plot("Systolic BP", shap_values, X, show=False)
# pl.xlim(80,225)
# pl.show()
# shap.dependence_plot("Poverty index", shap_values, X)

# shap.dependence_plot("White blood cells", shap_values, X, display_features=X_display, show=False)
# pl.xlim(2,15)
# pl.show()

In [18]:
# Neural network
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=0)
nn.fit(X_train, Y_train)
print_accuracy(nn.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(nn.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html


Accuracy = 53.833333333333336%


X does not have valid feature names, but MLPClassifier was fitted with feature names
Using 2400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/600 [00:00<?, ?it/s]

X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted

In [21]:
'''
# NN_gpt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd

# 定义神经网络模型
model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(Y.shape[1], activation='softmax'))

# 编译模型并指定损失函数和优化器
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练模型并评估性能
model.fit(X_train, Y_train, epochs=30, batch_size=10) # 300个测试数据
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
'''

IndexError: tuple index out of range

In [ ]:
'''
# MLP_MULI
# 与线性回归不同，MLP输出的是概率
import torch
from torch import nn
net = nn.Sequential(nn.Linear())
'''

In [36]:
# mlp模型性能评估
# 准确率
y_true = Y_test
y_pred_mlp = nn.predict(X_test)
accuracy_nn = accuracy_score(y_true, y_pred_mlp)

# 精确度
precision_nn = precision_score(y_true, y_pred_mlp)

# 召回率
recall_nn = recall_score(y_true, y_pred_mlp)

# F1分数
f1_nn = f1_score(y_true, y_pred_mlp)

# ROC-AUC分数
roc_auc_nn = roc_auc_score(y_true, y_pred_mlp)

print(accuracy_nn, precision_nn, recall_nn, f1_nn, roc_auc_nn)

0.5383333333333333 0.5387205387205387 0.5333333333333333 0.5360134003350082 0.5383333333333333


In [38]:
# 展示不同模型训练精度指标结果
print(accuracy_svm, precision_svm, recall_svm, f1_svm, roc_auc_svm) 
print(accuracy_lr, precision_lr, recall_lr, f1_lr, roc_auc_lr)
print(accuracy_dt, precision_dt, recall_dt, f1_dt, roc_auc_dt)
print(accuracy_rf, precision_rf, recall_rf, f1_rf, roc_auc_rf)
print(accuracy_xgb, precision_xgb, recall_xgb, f1_xgb, roc_auc_xgb)
print(accuracy_nn, precision_nn, recall_nn, f1_nn, roc_auc_nn)

0.73 0.73 0.73 0.7299999999999999 0.73
0.73 0.7363013698630136 0.7166666666666667 0.7263513513513512 0.7300000000000001
0.6916666666666667 0.6910299003322259 0.6933333333333334 0.692179700499168 0.6916666666666667
0.7883333333333333 0.7952218430034129 0.7766666666666666 0.7858347386172007 0.7883333333333333
0.7783333333333333 0.7755775577557755 0.7833333333333333 0.7794361525704809 0.7783333333333332
0.5383333333333333 0.5387205387205387 0.5333333333333333 0.5360134003350082 0.5383333333333333


In [37]:
# 要一个测试集和训练集的训练过程展示图_muli


0.73 0.73 0.73 0.7299999999999999 0.73
0.73 0.7363013698630136 0.7166666666666667 0.7263513513513512 0.7300000000000001
0.6916666666666667 0.6910299003322259 0.6933333333333334 0.692179700499168 0.6916666666666667
0.7883333333333333 0.7952218430034129 0.7766666666666666 0.7858347386172007 0.7883333333333333
0.7783333333333333 0.7755775577557755 0.7833333333333333 0.7794361525704809 0.7783333333333332
0.5383333333333333 0.5387205387205387 0.5333333333333333 0.5360134003350082 0.5383333333333333


In [ ]:
# 用性能最好的模型预测所有单元
# 绘制整个区域的易涝性，arcgis自然分割成多类


In [ ]:
# 建立一个GitHub仓库保存代码
# 应该是用vscode连接GitHub仓库进行代码的上传和更新
